In [21]:
import findspark
findspark.init('/opt/spark')

import os
from dotenv import load_dotenv
load_dotenv('../.env')
access = os.environ.get('AWS_ACCESS')
secret = os.environ.get('AWS_SECRET')

In [99]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import udf

from pyspark.ml.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.ml.feature import CountVectorizer , IDF

In [23]:
conf = SparkConf() \
    .set("fs.s3a.awsAccessKeyId", access) \
    .set("fs.s3a.awsSecretAccessKey", secret) \
    .set("fs.s3a.endpoint", "s3.us-east-1.amazonaws.com") \
    .set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .set("fs.s3a.impl","org.apache.hadoop.fs.s3native.NativeS3FileSystem") \
    .set("com.amazonaws.services.s3.enableV4", "true")

spark = SparkSession.builder.master('LDA').appName('cool').config(conf=conf).getOrCreate()

In [24]:
# filename = 's3a://patellism/processed_data/2020-08-cleaned.parquet.snappy/part-00004-d2d9c5cf-46de-47d9-86df-28fefd1709e5-c000.snappy.parquet'
filename = '2020-08-cleaned-small.snappy.parquet'
df = spark.read.parquet(filename).drop('geo', 'coordinates', 'place')
df.show(5)

+-------------------+--------------------+-------------+--------------+--------------------+
|                 id|           full_text|retweet_count|favorite_count|          clean_text|
+-------------------+--------------------+-------------+--------------+--------------------+
|1300070747059167233|@kmiranda1973 @Mi...|          0.0|           2.0|[kmiranda, militi...|
|1300070747453427713|@realDonaldTrump ...|          0.0|           0.0|[realdonaldtrump,...|
|1300070747566755845|@realDonaldTrump ...|          0.0|           1.0|[realdonaldtrump,...|
|1300070748116131840|@EricTrump @realD...|          0.0|           1.0|[erictrump, reald...|
|1300070748359454721|Impeached @realdo...|          0.0|           1.0|[impeach, realdon...|
+-------------------+--------------------+-------------+--------------+--------------------+
only showing top 5 rows



In [78]:
cv = CountVectorizer(inputCol='clean_text', outputCol='raw_features', vocabSize=5000, minDF=2)
print(cv.getVocabSize())
cvmodel = cv.fit(df.select('id', 'clean_text'))

result_cv = cvmodel.transform(df.select('id', 'clean_text'))

5000


In [79]:
idf = IDF(inputCol="raw_features", outputCol="features")
idfModel = idf.fit(result_cv)

result_tfidf = idfModel.transform(result_cv)

In [103]:
num_topics = 4
max_iterations = 50

lda = LDA(k=num_topics, maxIter=max_iterations)

model = lda.fit(result_tfidf)

In [104]:
topics = model.describeTopics(maxTermsPerTopic=5)
vocabArray = cvmodel.vocabulary

def covertToWord(indices):
    result = []
    for i in indices:
        result.append(vocabArray[i])
    return result

udf_convertToWord = udf(covertToWord, ArrayType(StringType()))
topics = topics.withColumn('word', udf_convertToWord('termIndices'))
topics.show(truncate=False)

+-----+-------------------+-----------------------------------------------------------------------------------------------------------+-----------------------------------------------------+
|topic|termIndices        |termWeights                                                                                                |word                                                 |
+-----+-------------------+-----------------------------------------------------------------------------------------------------------+-----------------------------------------------------+
|0    |[5, 13, 10, 27, 7] |[0.03750786836272506, 0.03143245359551565, 0.029592254015849327, 0.02862452470455506, 0.02833492061282447] |[know, president, need, start, law]                  |
|1    |[1, 22, 63, 47, 55]|[0.03184640017164536, 0.027572712045334968, 0.02444351086185667, 0.02332585668163815, 0.021549243717212806]|[joebiden, erictrump, donaldjtrumpjr, violence, look]|
|2    |[4, 19, 18, 44, 45]|[0.0540633495086206, 0.